## Import

In [1]:
# script pion.py hjyf
from tkinter import *
import tkinter as tk
import random
from functools import partial
import time
from mdpmadi22_class import Display
import numpy as np

# visualization

In [2]:

def create_Visualization(titre, env, zoom):
    env.globalcost= 0
    Mafenetre = Tk() 
    Mafenetre.title(titre)

    # def des couleurs
    myred="#D20B18"
    mygreen="#25A531"
    myblue="#0B79F7"
    mygrey="#E8E8EB"
    myyellow="#F9FB70"
    myblack="#2D2B2B"
    mywalls="#5E5E64"
    mywhite="#FFFFFF"
    color=[mywhite,mygreen,myblue,myred,myblack,mygrey]
    # Creation d'un widget Canvas (pour la grille)
    Largeur = zoom*20*env.nb_columns+40
    Hauteur = zoom*20*env.nb_lines+40
    # ecriture du quadrillage et coloration
    Canevas = Canvas(Mafenetre, width = Largeur, height =Hauteur, bg =mywhite)
    # cout et affichage
    
    for i in range(env.nb_lines+1):
        ni=zoom*20*i+20
        Canevas.create_line(20, ni, Largeur-20,ni)
    for j in range(env.nb_columns+1):
        nj=zoom*20*j+20
        Canevas.create_line(nj, 20, nj, Hauteur-20)
    colordraw(env, color, Canevas,zoom)

    
  

    # Creation d'un widget Button (bouton Quitter)
    Button(Mafenetre, text ='Restart', command = create_Visualization).pack(side=LEFT,padx=5,pady=5)
    Button(Mafenetre, text ='Quit', command = Mafenetre.destroy).pack(side=LEFT,padx=5,pady=5)

    w = Label(Mafenetre, text='Cost = '+str(env.globalcost),fg=myblack,font = "Verdana 14 bold")
    w.pack() 

    Pion = Canevas.create_oval(env.pos_x-10,env.pos_y-10,env.pos_x+10,env.pos_y+10,width=2,outline='black',fill=myyellow)
    Canevas.coords(Pion,env.pos_x -9*zoom, env.pos_y -9*zoom, env.pos_x +9*zoom, env.pos_y +9*zoom)
    w.config(text='Cost = '+ str(env.globalcost))
    process_Clavier = partial(Clavier, env=env, Canevas=Canevas, zoom=zoom, Pion=Pion,w=w)
    Canevas.focus_set()
    Canevas.bind('<Key>',process_Clavier)
    Canevas.pack(padx =5, pady =5)

    env.pos_x = 20+10*zoom
    env.pos_y = 20+10*zoom
    return Mafenetre
# specification des proportion de murs, case blanches et pts de couleur
def colordraw(env, color, Canevas, zoom):

    
    for i in range(env.nb_lines):
        for j in range(env.nb_columns):          
            y =zoom*20*i+20
            x =zoom*20*j+20
            if env.g[i,j]>0:
                # print(i,j)
                # print(env.g)
                # print(color[env.g[i,j]])            
                Canevas.create_oval(x+zoom*(10-3),y+zoom*(10-3),x+zoom*(10+3),y+zoom*(10+3),width=1,outline=color[env.g[i,j]],fill=color[env.g[i,j]])
            else:
                if env.g[i,j]<0:
                    Canevas.create_rectangle(x, y, x+zoom*20, y+zoom*20, fill=color[4])
                    Canevas.create_rectangle(x, y, x+zoom*20, y+zoom*20, fill=color[4])
    #ajoute le but
    
    
    y =zoom*20*env.iBut+20
    x =zoom*20*env.jBut+20
    Canevas.create_rectangle(x, y, x+zoom*20, y+zoom*20, fill=color[1])
    Canevas.create_rectangle(x, y, x+zoom*20, y+zoom*20, fill=color[1])

  
def Clavier(event, env, Canevas, zoom, Pion,w):
    
    touche = event.keysym
    posebase = (env.pos_x,env.pos_y)
    cj=int((env.pos_x-30)/(20*zoom))
    li=int((env.pos_y-30)/(20*zoom))
    changed=0
    cost = np.zeros(6,dtype=np.int)
    for k in range(6):
        cost[k]=0
    # deplacement aleatoire en appuyant sur space
    if touche == 'space':
        

        # t=np.random.randint(6)
        lettre = ['f','g','h','j','y','u',]
        
        touche=env.execute_politique()
        
    # deplacement (-2,1)
    if touche == 'y' and li>1 and cj < env.nb_columns-1 and env.g[li-2,cj+1]>-1:
        env.pos_y -= zoom*20*2
        env.pos_x += zoom*20 
        cost[env.g[li-2,cj+1]]+=1 
        changed=1
    # deplacement (-2,-1)
    if touche == 't' and li>1 and cj > 0 and env.g[li-2,cj-1]>-1:
        env.pos_y -= zoom*20*2       
        env.pos_x -= zoom*20                 
        cost[env.g[li-2,cj-1]]+=1
        changed=1
   # deplacement (-1,2)
    if touche == 'u' and li>0 and cj < env.nb_columns-2 and env.g[li-1,cj+2]>-1:
        env.pos_y -= zoom*20        
        env.pos_x += zoom*20*2       
        cost[env.g[li-1,cj+2]]+=1
        changed=1
    # deplacement (-1,-2)
    if touche == 'r' and li>0 and cj >1 and env.g[li-1,cj-2]>-1:
        env.pos_y -= zoom*20
        env.pos_x -= zoom*20*2           
        cost[env.g[li-1,cj-2]]+=1
        changed=1
     # deplacement (2,1)  
    if touche == 'h' and li<env.nb_lines-2 and cj < env.nb_columns-1 and env.g[li+2,cj+1]>-1:
        env.pos_y += zoom*20*2
        env.pos_x += zoom*20 
        cost[env.g[li+2,cj+1]]+=1
        changed=1
    # deplacement (2,-1)
    if touche == 'g' and li<env.nb_lines-2 and cj > 0 and env.g[li+2,cj-1]>-1:
        env.pos_y += zoom*20*2       
        env.pos_x -= zoom*20                 
        cost[env.g[li+2,cj-1]]+=1
        changed=1
   # deplacement (1,2)
    if touche == 'j' and li<env.nb_lines-1 and cj < env.nb_columns-2 and env.g[li+1,cj+2]>-1:
        env.pos_y += zoom*20        
        env.pos_x += zoom*20*2       
        cost[env.g[li+1,cj+2]]+=1
        changed=1
    # deplacement (1,-2)
    if touche == 'f' and li<env.nb_lines-1 and cj >1 and env.g[li+1,cj-2]>-1:
        env.pos_y += zoom*20
        env.pos_x -= zoom*20*2           
        cost[env.g[li+1,cj-2]]+=1 
        changed=1

    newcj=int((env.pos_x-30)/(20*zoom))
    newli=int((env.pos_y-30)/(20*zoom))
    env.posCav=(newli,newcj)

# La variable alea =1 si on veut des effets aleatoires sinon les transitions sont deterministes
    #On ajoute un effet aleatoire dans les transitions
    if env.alea==1 and changed==1:
        t=np.random.uniform(0,1)    
        if t>0.5:
            d=np.random.randint(8)
            dli=0
            if d== 0 or d==1 or d==2:
                dli=-1
            if d== 4 or d==5 or d==6:
                dli==1
            dcj=0
            if d==0 or d==7 or d==6:
                dcj=-1
            if d==2 or d==3 or d==4:
                dcj=1    
        # l'effet aleatoire est applique s'il cree un deplacement sur une case admissible  
            # dli=1
            # dcj=0   
            Newpos_y = env.pos_y+zoom*20*dli
            Newpos_x = env.pos_x+zoom*20*dcj        
            newcj=int((Newpos_x-30)/(20*zoom))
            newli=int((Newpos_y-30)/(20*zoom))

            # print('d',dli,dcj)
            if newli>=0 and newcj>=0 and newli<=env.nb_lines-1 and newcj<=env.nb_columns-1 and env.g[newli,newcj]>-1:
                env.pos_y=Newpos_y
                env.pos_x=Newpos_x            
                env.posCav=(newli,newcj)
                cost = np.zeros(6,dtype=np.int)
                cost[env.g[newli,newcj]]+=1 

    Canevas.create_line(posebase[0], posebase[1], env.pos_x , env.pos_y, arrow=tk.LAST)
# on dessine le pion a sa nouvelle position
    Canevas.coords(Pion,env.pos_x -9*zoom, env.pos_y -9*zoom, env.pos_x +9*zoom, env.pos_y +9*zoom)       
    # env.globalcost=0    
    for k in range(6):
        env.globalcost+=cost[k]*env.weight[k]
    w.config(text='Cost = '+ str(-env.globalcost))

# Structure 

In [3]:
class state:
    
    

    def __init__(self, coords, Value, reward, dictNStates, dictAroundStates, isReward):
        
        self.coords = coords
        self.value = Value
        self.reward = reward
        self.bestAction = ' '
        self.IsaReward = isReward
        self.dictNextStates = dictNStates
        self.nbNextStates = len(self.dictNextStates)
        self.dictAroundStates = dictAroundStates
        self.nbStatesAround = len(dictAroundStates)
        self.Probability = 1 - self.nbStatesAround/16 

    
        
    def print_state(self):

        print("coord: ",self.coords)
        print("value: ",self.value)
        print("reward: ",self.reward)
        print("dictNStates: ",self.dictNextStates)
        print("dictAroundStates: ",self.dictAroundStates)
        print("nbStatesAround: ",self.nbStatesAround)

In [4]:
class env:
    def __init__(self, nb_columns, nb_lines, weight, alea, PmurDict, seed, posCav):
        self.nb_columns = nb_columns
        self.nb_lines = nb_lines
        self.g = np.zeros((nb_lines,nb_columns),dtype=np.int)
        self.weight = weight

        
        # self.zoom = zoom
        self.alea = alea
        self.global_cost = 0
        self.p_tile_type = PmurDict
        self.seed = seed
        self.posCav = posCav
        self.pos_x =20
        self.pos_y =20
        self.genere_map_alea()
        self.listAllStates = self.create_all_states()

    def genere_map_alea(self):
        np.random.seed(self.seed)
        
        
        for i in range(self.nb_lines):
            for j in range(self.nb_columns):
                z = np.random.uniform(0, 1)
                if z < self.p_tile_type['pwall']:
                    c = -1
                elif z < self.p_tile_type['pwall'] + self.p_tile_type['pwhite']:
                    c = 0
                elif z < self.p_tile_type['pwall'] + self.p_tile_type['pwhite'] + self.p_tile_type['pgreen']:
                    c = 1
                elif z < self.p_tile_type['pwall'] + self.p_tile_type['pwhite'] + self.p_tile_type['pgreen'] + self.p_tile_type['pblue']:
                    c = 2
                elif z < self.p_tile_type['pwall'] + self.p_tile_type['pwhite'] + self.p_tile_type['pgreen'] + self.p_tile_type['pblue'] + self.p_tile_type['pred']:
                    c = 3
                else:
                    c = 4
                self.g[i, j] = c  
                self.g[i,j]=c
        self.g[0,0]=0
        self.g[0,1]=0
        self.g[2,0]=0    
        self.g[self.nb_lines-1,self.nb_columns-1]=0
        self.g[self.nb_lines-2,self.nb_columns-1]=0
        self.g[self.nb_lines-1,self.nb_columns-2]=0
        self.iBut = np.random.randint(2, self.nb_lines-1)
        self.jBut = np.random.randint(2, self.nb_columns-1)
        self.g[self.iBut,self.jBut] = 5        

    def create_all_states(self):
        l = list()
        for i in range(self.nb_lines):
            for j in range(self.nb_columns):
                coord =  (i,j)
                isaReward = False
                test = True
                if self.g[i,j] < 0:
                    reward  = " mur "
                    test = False
                else:
                    #reward = np.format_float_scientific(self.weight[env.g[i,j]],exp_digits=1)
                    reward = -self.weight[self.g[i,j]]
                    if reward > 0: 
                        isaReward = True
                if test == True:        
                    value = reward
                else :
                    value = 0
                dictNStates = self.get_dictNextStates(i, j)
                dictAroundStates = self.get_around_States_Proba(i, j)
                
                
                s = state(coord, value, reward, dictNStates, dictAroundStates,isaReward )
                l.append(s)
        return l
    def get_dictNextStates(self,i,j):
        #check OutOfBound
        di = dict()
        if i-2>=     0       and j+1 <self.nb_columns and self.g[i-2,j+1] != -1:
            di['y']=(i-2,j+1)
        if i-1>=     0       and j+2 <self.nb_columns and self.g[i-1,j+2] != -1:
            di['u']=(i-1,j+2)
        if i+1<self.nb_lines and j+2 <self.nb_columns and self.g[i+1,j+2] != -1:
            di['j']=(i+1,j+2)
        if i+2<self.nb_lines and j+1 <self.nb_columns and self.g[i+2,j+1] != -1:
            di['h']=(i+2,j+1)
        if i+2<self.nb_lines and j-1 >=       0       and self.g[i+2,j-1] != -1:
            di['g']=(i+2,j-1)
        if i+1<self.nb_lines and j-2 >=       0       and self.g[i+1,j-2] != -1:
            di['f']=(i+1,j-2)
        if i-1>=     0       and j-2 >=       0       and self.g[i-1,j-2] != -1:
            di['r']=(i-1,j-2)
        if i-2>=     0       and j-1 >=       0       and self.g[i-2,j-1] != -1:
            di['t']=(i-2,j-1)

        return di
    def get_around_States_Proba(self, i, j):
        # check OutOfBound
        diStateAround = dict()
        if i+1<self.nb_lines and j   <self.nb_columns and self.g[i+1,j  ] != -1:
            diStateAround['md'] = (i+1,j  , 1/16)

        if i-1>=      0      and j   <self.nb_columns and self.g[i-1,j  ] != -1:
            diStateAround['mg'] = (i-1,j  , 1/16)

        if i+1<self.nb_lines and j+1 <self.nb_columns and self.g[i+1,j+1] != -1:
            diStateAround['hd'] = (i+1,j+1, 1/16)

        if i-1>=      0      and j-1 >=      0        and self.g[i-1,j-1] != -1:
            diStateAround['bg'] = (i-1,j-1, 1/16)

        if i+1<self.nb_lines and j-1 >=      0        and self.g[i+1,j-1] != -1:
            diStateAround['bd'] = (i+1,j-1, 1/16)

        if i-1>=      0      and j+1 <self.nb_columns and self.g[i-1,j+1] != -1:
            diStateAround['hg'] = (i-1,j+1, 1/16)

        if i <self.nb_lines  and j+1 <self.nb_columns and self.g[i  ,j+1] != -1:
            diStateAround['hm'] = (i  ,j+1, 1/16)

        if i <self.nb_lines  and j-1 >=      0        and self.g[i  ,j-1] != -1:
            diStateAround['bm'] = (i  ,j-1, 1/16)
            
        return diStateAround

    def execute_politique(self):
        
        return self.get_state(self.posCav[0],self.posCav[1]).bestAction 

    def get_state(self,i,j):
        i = int(i)
        j = int(j)
        if i>=0 and i<self.nb_lines and j >=0 and j<self.nb_columns:
            return self.listAllStates[i*self.nb_columns+j]
        return None

    def get_previous_states(self, s):
        l = list()
        for sp in self.listAllStates:
            if s.coords in sp.dictNextStates.values():
                l.append(sp)
        if self.alea == 1:
            neighbors = self.get_around_States_Proba(s.coords[0], s.coords[1])
            for neighbor in neighbors.values():
                for sp in self.listAllStates:
                    if (neighbor[0], neighbor[1]) in sp.dictNextStates.values():
                        l.append(sp)
        return l
    
    def T(self, s, a, sp):
        if s.dictNextStates[a] == sp.coords:
            if self.alea == 0:
                return 1
            else:
                return sp.Probability
        else:
            return 0
    def afficheV(self):
        print("Valeur de l'etat selon l'algo")
        for i in range(self.nb_lines):
            l = list()
            for j in range(self.nb_columns):
                value="{:.2e}".format(self.get_state(i,j).value)
                l.append(value)
            print(l)
          

    def afficheBestAction(self):
        print("meilleures Actions")
        for i in range(self.nb_lines):
            l = list()
            for j in range(self.nb_columns):
                l.append(self.get_state(i,j).bestAction)
            print(l)

    def afficheP(self):
        print("Proba d'arriver ce State")
        for i in range(self.nb_lines):
            l = list()
            for j in range(self.nb_columns):
                l.append(self.get_state(i,j).Probability)
            print(l)

    def afficheNBSA(self):
        print("NB State Around")
        for i in range(self.nb_lines):
            l = list()
            for j in range(self.nb_columns):
                l.append(self.get_state(i,j).nbStatesAround)
            print(l)

    def afficheR(self):
        print("Rewards")
        for i in range(self.nb_lines):
            l = list()
            for j in range(self.nb_columns):
                l.append(self.get_state(i,j).reward)
            print(l)
    def afficheNBNS(self):
        print("NB Next State ")
        for i in range(self.nb_lines):
            l = list()
            for j in range(self.nb_columns):
                l.append((self.get_state(i,j).nbNextStates,self.get_state(i,j).dictNextStates))
            print(l)



### verification

In [5]:
zoom = 2

#transitions aleatoires si alea =1 sinon mettre alea=0
alea = 0 
#taille de la grille
nb_lines = 10
nb_columns = 20 



# valeurs de la grille
weight = np.zeros(6,dtype=np.int)
weight[0] = 1
weight[1] = 2
weight[2] = 4
weight[3] = 8
weight[4] = 16
weight[5] = -1000

seed = 0

PmurDict=dict()
PmurDict['pwall'] = 0.15 
PmurDict['pwhite'] = 0.45 
PmurDict['pgreen'] = 0.1
PmurDict['pblue'] = 0.1
PmurDict['pred'] = 0.1
#  pnoire=0.1 mais pas besoin de le specifier c'est la couleur restante
        #i,j
poscav = (0,0)
e = env(nb_columns,nb_lines,weight=weight,alea=alea,PmurDict=PmurDict, seed=seed, posCav=poscav)
e.genere_map_alea()
print(e.g)
# e.afficheV()
# e.afficheP()
# e.afficheNBSA()
# e.afficheR()
e.afficheBestAction()
# e.afficheNBNS() 
Mafenetre = create_Visualization('test',e, zoom)
Mafenetre.mainloop()

/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_24613/1899825460.py:12: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  weight = np.zeros(6,dtype=np.int)
/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_24613/1732703606.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curren

[[ 0  0  1  0  0  1  0  3  4  0  2  0  0  4 -1 -1 -1  3  2  3]
 [ 4  2  0  2 -1  1 -1  4  0  0  0  2  0  0 -1  1  1  1  4  1]
 [ 0  0  1 -1  1  1  0 -1  0  0  0  0  4 -1  0  0  1  0  0  0]
 [ 0 -1  1 -1  0  0  3 -1  3 -1  4  0  4  1  2 -1  0 -1  0 -1]
 [ 0  0 -1  1  0  0  0 -1  0  4  0  1 -1  2  0  0  0 -1  3 -1]
 [ 1  0  2  4  0  0  0  0  0  4  0  3  1  0  3  0  3  0  3  1]
 [ 2  0  4  1  0  1 -1  0  1  0  1  0 -1  0  0  0  0  1  1  0]
 [ 3  0  0  3  3  2 -1  4  2  4 -1  3  0  1 -1  3  3  0  0 -1]
 [ 1  0  2  3  4  3 -1  0  2  5  0 -1  0 -1  2  0  0  4  2  0]
 [ 0  1  0  0  4  1  0  0  2  0  0  0  0  4  2  0  0  0  0  0]]
meilleures Actions
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '

/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_24613/3022147063.py:85: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cost = np.zeros(6,dtype=np.int)


# Partie 1 Recherche d'un trajectoire prudente

## question A
 Modéliser le problème comme un processus décisionnel Markovien en donnant une récompense significative lorsqu’on atteint la case but, par exemple un bonus (réduction de coût) de 1000 points mais on pourra essayer d’autres valeurs en fonction de la dimension de la grille choisie. Ecrire les équations de Bellman définissant la valeur  vij d’être dans une case (i, j) pour une politique stationnaire optimale et un facteur d’actualisation γ.

## question B
Dans un premier temps on se place dans le cas d éterministe (la case cible est atteinte avec une probabilité 1 à chaque saut). Déterminer par itération de la valeur un chemin optimal de la case initiale vers la case but et tester sur une grille de taille puis 5×10, 10×15, 15×20 (on essaiera avec γ = 1 et γ = 0.5). On donnera les grilles en visualisant les trajectoires optimales et on donnera les temps de calcul.

### Code

In [6]:
def value_iterationD(env, gamma, epsilon):
    Value_states = np.zeros(env.nb_lines*env.nb_columns, dtype=np.int)
    e = env

    iteration = 0
    stop = False
    while stop==False:
        stop=True
        for count, s in enumerate(e.listAllStates):
            best = float('-inf')
            saveA = ' '
            
            if s.reward == " mur ":
                continue
            elif s.IsaReward == True:
                
                s.bestAction = 'R'
                continue
            for a, v in s.dictNextStates.items():
                pState = e.get_state(v[0],v[1]) 
                
                temp = s.reward+gamma*pState.value 

                if temp > best:
                    best = temp
                    saveA = a
            s.bestAction = saveA
            if abs(s.value - best) > epsilon:
                        stop=False
            s.value = best
        iteration +=1

        
       
        if iteration>100 :
            stop = True
    print(iteration)

### TEST

In [7]:
#transitions aleatoires si alea =1 sinon mettre alea=0
alea = 0 
#taille de la grille
nb_lines = 5
nb_columns = 10

#cost = np.zeros(6,dtype=np.int)

# valeurs de la grille
weight = np.zeros(6,dtype=np.int)
weight[0] = 1
weight[1] = 2
weight[2] = 4
weight[3] = 8
weight[4] = 16
weight[5] = -1000

seed = 0
PmurDict=dict()
PmurDict['pwall'] = 0.15 
PmurDict['pwhite'] = 0.45 
PmurDict['pgreen'] = 0.1
PmurDict['pblue'] = 0.1
PmurDict['pred'] = 0.1
#  pnoire=0.1 mais pas besoin de le specifier c'est la couleur restante
poscav=(0,0)
e = env(nb_columns,nb_lines,weight=weight,alea=alea,PmurDict=PmurDict, seed=seed, posCav=poscav)
e.genere_map_alea()

gamma = 1
epsilon = 1

debut = time.perf_counter()
value_iterationD(env=e, gamma= gamma, epsilon=epsilon)
fin = time.perf_counter()
print("Temps de calcul :", fin - debut, "secondes")
zoom = 2

display = Display(env=e, zoom=zoom)
display.initialize()
display.run()

# e.afficheBestAction()
# Mafenetre = create_Visualization('test',e, zoom)
# Mafenetre.mainloop()


/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_24613/1289049367.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  weight = np.zeros(6,dtype=np.int)
/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_24613/1732703606.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curren

5
Temps de calcul : 0.0010745000035967678 secondes
<class 'int'>
<class 'int'>
(40, 40)
(120, 80)
(160, 160)
(80, 200)
(120, 120)


In [8]:
#taille de la grille
nb_lines = 10
nb_columns = 15

e = env(nb_columns,nb_lines,weight=weight,alea=alea,PmurDict=PmurDict, seed=seed, posCav=poscav)
e.genere_map_alea()

gamma = 1
epsilon = 1

debut = time.perf_counter()
value_iterationD(env=e, gamma= gamma, epsilon=epsilon)
fin = time.perf_counter()
print("Temps de calcul :", fin - debut, "secondes")


# e.afficheBestAction()
# Mafenetre = create_Visualization('test',e, zoom)
# Mafenetre.mainloop()
display = Display(env=e, zoom=zoom)
display.initialize()
display.run()

/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_24613/1732703606.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.g = np.zeros((nb_lines,nb_columns),dtype=np.int)
/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_24613/3075554346.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to

7
Temps de calcul : 0.0037397090054582804 secondes
meilleures Actions
['h', 'h', 'h', 'g', 'h', 'j', 'h', 'g', 'j', 'h', 'h', 'j', 'j', 'h', ' ']
[' ', ' ', 'h', 'h', 'g', 'j', 'g', 'u', 'j', ' ', 'g', ' ', 'h', 'g', 'g']
['y', 'j', 'j', 'j', ' ', 'u', 'g', 'j', 'j', 'j', 'j', 'u', 'g', ' ', 'f']
['h', 'u', ' ', 'j', 'g', 'u', 'y', 'f', ' ', 'j', 'h', 'j', 'g', 'f', 'g']
['h', ' ', 'u', ' ', 'j', 'h', 'y', ' ', 'u', ' ', 'u', 'f', 'g', 'f', 'r']
[' ', 'h', ' ', 'u', ' ', 'y', 'g', ' ', 'y', 'h', 't', 'g', ' ', 'g', 't']
['j', 'u', ' ', 'j', 'y', 'g', 'j', ' ', 'j', ' ', 'y', 'r', 'f', 'r', 'f']
['y', 'j', 'j', 'u', 'u', 'h', 'j', 'g', 'y', 'u', 'R', 'j', 't', 'r', 'g']
['u', 'u', 'j', 'u', 'j', 'y', ' ', 't', 'u', 'f', 'y', 'j', ' ', 'y', 'r']
['y', 'y', 'u', 'u', 'y', 'u', 'u', 'y', 'u', 'y', 'r', ' ', 'u', 't', 't']


In [9]:
#taille de la grille
nb_lines = 15
nb_columns = 20

e = env(nb_columns,nb_lines,weight=weight,alea=alea,PmurDict=PmurDict, seed=seed, posCav=poscav)
e.genere_map_alea()

gamma = 1
epsilon = 1

debut = time.perf_counter()
value_iterationD(env=e, gamma= gamma, epsilon=epsilon)
fin = time.perf_counter()
print("Temps de calcul :", fin - debut, "secondes")

# e.afficheBestAction()
# Mafenetre = create_Visualization('test',e, zoom)
# Mafenetre.mainloop()
display = Display(env=e, zoom=zoom)
display.initialize()
display.run()

/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_24613/1732703606.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.g = np.zeros((nb_lines,nb_columns),dtype=np.int)
/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_24613/3075554346.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to

8
Temps de calcul : 0.011990874976618215 secondes
meilleures Actions
['j', 'h', 'g', 'j', 'h', 'h', 'j', 'j', 'h', 'h', 'h', 'j', 'g', 'h', ' ', ' ', ' ', 'h', 'g', 'g']
['j', 'h', 'u', 'h', ' ', 'g', ' ', 'j', 'j', 'j', 'u', 'f', 'j', 'j', ' ', 'h', 'j', 'g', 'f', 'g']
['j', 'g', 'j', ' ', 'h', 'h', 'g', ' ', 'u', 'h', 'h', 'g', 'g', ' ', 'j', 'g', 'g', 'g', 'f', 'r']
['u', ' ', 'j', ' ', 'j', 'h', 'j', ' ', 'j', ' ', 'f', 'r', 'j', 'u', 'h', ' ', 'g', ' ', 'g', ' ']
['u', 'y', ' ', 'u', 'j', 'j', 'j', ' ', 'y', 'h', 'h', 'j', ' ', 'j', 'g', 'g', 'g', ' ', 'h', ' ']
['h', 'h', 'u', 'y', 'u', 'j', 'j', 'j', 'u', 'j', 'f', 'j', 'u', 'g', 'g', 'f', 'h', 'j', 'g', 'g']
['j', 'g', 'u', 'y', 'u', 'y', ' ', 'y', 'h', 'y', 'j', 'h', ' ', 'g', 'f', 'j', 'j', 'g', 'f', 'f']
['y', 'u', 'u', 'y', 'h', 'j', ' ', 'j', 'j', 'u', ' ', 'u', 'g', 'g', ' ', 'h', 'g', 'g', 'h', ' ']
['u', 'j', 'h', 'y', 'j', 'j', ' ', 'j', 'j', 'j', 'y', ' ', 'h', ' ', 'g', 'u', 'h', 'f', 'g', ' ']
['j', 'y', 'h', 'h', '

In [10]:
#taille de la grille
nb_lines = 5
nb_columns = 10

e = env(nb_columns,nb_lines,weight=weight,alea=alea,PmurDict=PmurDict, seed=seed, posCav=poscav)
e.genere_map_alea()

gamma = 0.5
epsilon = 1

debut = time.perf_counter()
value_iterationD(env=e, gamma= gamma, epsilon=epsilon)
fin = time.perf_counter()
print("Temps de calcul :", fin - debut, "secondes")

# e.afficheBestAction()
# Mafenetre = create_Visualization('test',e, zoom)
# Mafenetre.mainloop()
display = Display(env=e, zoom=zoom)
display.initialize()
display.run()

/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_24613/1732703606.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.g = np.zeros((nb_lines,nb_columns),dtype=np.int)
/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_24613/3075554346.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to

5
Temps de calcul : 0.0015429169870913029 secondes
meilleures Actions
['h', 'h', 'f', 'g', 'h', 'f', 'g', 'j', 'g', 'f']
['j', 'u', 'h', 'h', ' ', ' ', ' ', 'f', 'r', 'g']
['y', 'h', 'j', 'g', ' ', 'h', ' ', 'g', 'h', 'f']
['u', 't', 'y', 'f', 'R', 'r', 'f', 'r', 'f', 'r']
['y', 'y', 'u', ' ', 'y', 'r', 'r', ' ', 't', 'r']


In [11]:
#taille de la grille
nb_lines = 10
nb_columns = 15

e = env(nb_columns,nb_lines,weight=weight,alea=alea,PmurDict=PmurDict, seed=seed, posCav=poscav)
e.genere_map_alea()

gamma = 0.5
epsilon = 1

debut = time.perf_counter()
value_iterationD(env=e, gamma= gamma, epsilon=epsilon)
fin = time.perf_counter()
print("Temps de calcul :", fin - debut, "secondes")

# e.afficheBestAction()
# Mafenetre = create_Visualization('test',e, zoom)
# Mafenetre.mainloop()
display = Display(env=e, zoom=zoom)
display.initialize()
display.run()

/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_24613/1732703606.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.g = np.zeros((nb_lines,nb_columns),dtype=np.int)
/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_24613/3075554346.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to

6
Temps de calcul : 0.0053073749877512455 secondes
meilleures Actions
['h', 'h', 'h', 'g', 'g', 'j', 'h', 'h', 'j', 'h', 'h', 'j', 'j', 'h', ' ']
[' ', ' ', 'h', 'j', 'h', 'h', 'g', 'u', 'j', ' ', 'g', ' ', 'h', 'g', 'g']
['y', 'j', 'j', 'j', ' ', 'g', 'g', 'j', 'j', 'j', 'j', 'u', 'g', ' ', 'f']
['h', 'u', ' ', 'j', 'h', 'h', 'f', 'g', ' ', 'j', 'h', 'j', 'g', 'f', 'g']
['h', ' ', 'y', ' ', 'j', 'h', 'g', ' ', 'u', ' ', 'u', 'f', 'g', 'f', 'r']
[' ', 'h', ' ', 'u', ' ', 'h', 'j', ' ', 'y', 'h', 'j', 'g', ' ', 'g', 'f']
['j', 'u', ' ', 'j', 'u', 'g', 'j', ' ', 'j', ' ', 'y', 'r', 'f', 'r', 'f']
['y', 'j', 'j', 'u', 'u', 'h', 'j', 'g', 'y', 'u', 'R', 'j', 't', 'r', 'r']
['u', 'u', 'j', 'u', 'j', 'y', ' ', 'y', 'u', 't', 'y', 'y', ' ', 't', 'r']
['y', 'y', 'u', 't', 'y', 'u', 'u', 'y', 'u', 'y', 'r', ' ', 'u', 't', 't']


In [12]:
#taille de la grille
nb_lines = 15
nb_columns = 20

e = env(nb_columns,nb_lines,weight=weight,alea=alea,PmurDict=PmurDict, seed=seed, posCav=poscav)
e.genere_map_alea()

gamma = 0.5
epsilon = 1

debut = time.perf_counter()
value_iterationD(env=e, gamma= gamma, epsilon=epsilon)
fin = time.perf_counter()
print("Temps de calcul :", fin - debut, "secondes")

# e.afficheBestAction()
# e.afficheV()
# Mafenetre = create_Visualization('test',e, zoom)
# Mafenetre.mainloop()
display = Display(env=e, zoom=zoom)
display.initialize()
display.run()

/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_24613/1732703606.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.g = np.zeros((nb_lines,nb_columns),dtype=np.int)
/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_24613/3075554346.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to

8
Temps de calcul : 0.012219500000355765 secondes
meilleures Actions
['j', 'g', 'g', 'j', 'h', 'h', 'j', 'j', 'h', 'h', 'h', 'j', 'g', 'h', ' ', ' ', ' ', 'h', 'g', 'g']
['j', 'h', 'u', 'h', ' ', 'g', ' ', 'j', 'j', 'j', 'u', 'f', 'j', 'j', ' ', 'h', 'j', 'g', 'f', 'g']
['j', 'g', 'j', ' ', 'h', 'h', 'j', ' ', 'h', 'h', 'h', 'g', 'g', ' ', 'j', 'g', 'g', 'g', 'f', 'r']
['h', ' ', 'j', ' ', 'j', 'h', 'j', ' ', 'j', ' ', 'h', 'r', 'j', 'u', 'h', ' ', 'g', ' ', 'g', ' ']
['u', 'j', ' ', 'u', 'j', 'j', 'j', ' ', 'y', 'h', 'h', 'j', ' ', 'j', 'g', 'g', 'g', ' ', 'h', ' ']
['h', 'h', 'u', 'j', 'u', 'j', 'j', 'j', 'u', 'j', 'f', 'j', 'u', 'g', 'u', 'f', 'h', 'f', 'g', 'g']
['j', 'j', 'u', 'y', 'u', 'j', ' ', 'h', 'h', 'y', 'j', 'h', ' ', 'g', 'f', 'g', 'h', 'g', 'f', 'f']
['y', 'u', 'u', 'j', 'y', 'j', ' ', 'j', 'j', 'u', ' ', 'u', 'g', 'g', ' ', 'h', 'g', 'g', 'h', ' ']
['u', 'j', 'h', 'y', 'j', 'j', ' ', 'j', 'j', 'j', 'y', ' ', 'h', ' ', 'g', 'u', 'h', 'f', 'g', ' ']
['j', 'y', 'h', 'u', '

## question C
On se place maintenant dans le cas de transitions aléatoires définies comme expliqué ci-dessus. On souhaite déterminer une politique optimale par itération de la valeur et effectuer des essais numériques de résolution de grilles de différentes tailles. Pour chaque taille on tirera 10 instances de grilles aléatoirement et on donnera dans un tableau le temps moyen de résolution et le nombre moyen d’itérations. On pourra également étudier l’impact de γ en faisant varier γ = 0.9, γ = 0.7, γ = 0.5.

### Code

In [13]:
def value_iterationG(env, gamma, epsilon):
    Value_states = np.zeros(env.nb_lines*env.nb_columns, dtype=np.int)
    e = env

    iteration = 0
    stop = False
    while stop==False:
        stop=True
        for count, s in enumerate(e.listAllStates):
            best = float('-inf')
            saveA = ' '
            
            if s.reward == " mur ":
                continue
            elif s.IsaReward == True:
                
                s.bestAction = 'R'
                continue
            for a, v in s.dictNextStates.items():
                somme = 0
                pState = e.get_state(v[0],v[1]) 
                somme = pState.Probability*pState.value 
                #print(pState.Probability)             
                for cState in pState.dictAroundStates.values():
                    
                    sA = e.get_state(cState[0],cState[1])
                    somme+= (1/16*sA.value)
                        
                temp = s.reward+gamma*somme

                if temp > best:
                    best = temp
                    saveA = a
            s.bestAction = saveA
            if abs(s.value - best) > epsilon:
                        stop=False
            s.value = best
        iteration +=1
        # print(iteration)
        # e.afficheBestAction()
        if iteration>1000 :
            stop = True
    return iteration

### TEST

In [14]:

#transitions aleatoires si alea =1 sinon mettre alea=0
alea = 1
#taille de la grille
nb_lines = 50
nb_columns = 50

#cost = np.zeros(6,dtype=np.int)

# valeurs de la grille

weight = np.zeros(6,dtype=np.int)
weight[0] = 1
weight[1] = 2
weight[2] = 4
weight[3] = 8
weight[4] = 16
weight[5] = -1000

seed = 0
PmurDict=dict()
PmurDict['pmur'] = 0.15 
PmurDict['pblanc'] = 0.45 
PmurDict['pverte'] = 0.1
PmurDict['pbleue'] = 0.1
PmurDict['prouge'] = 0.1
#  pnoire=0.1 mais pas besoin de le specifier c'est la couleur restante
poscav=(0,0)
e = env(nb_columns,nb_lines,weight=weight,alea=alea,PmurDict=PmurDict, seed=seed, posCav=poscav)
e.genere_map_alea()
print(e.g)
gamma = 0.3
epsilon = 1

value_iterationG(env=e, gamma= gamma, epsilon=epsilon)
Mafenetre = create_Visualization('test',e, zoom)
Mafenetre.mainloop()
print(e.globalcost)

def experience(seed,list_gamma,list_taille):
    random.seed(seed)
    
    # list_gamma = [0.5]
    # list_taille = [(5,5)]
    result = np.zeros((len(list_gamma),len(list_taille),10,2),dtype=float)
    for cg, gamma in enumerate(list_gamma):
        for ct, taille in enumerate(list_taille):
            for i in range (10):
                
                seedA = random.randint(1, 100)

                e = env(taille[0],taille[1],weight=weight,alea=alea,PmurDict=PmurDict, seed=seedA, posCav=poscav)
                e.genere_map_alea()
                debut = time.perf_counter()
                it = value_iterationG(env=e, gamma= gamma, epsilon=epsilon)
                fin = time.perf_counter()
                temps  = fin - debut
                print(temps,it)
                result[cg,ct,i,0] = it
                result[cg,ct,i,1] = temps
    return result
  
# Mafenetre.destroy()

/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_24613/949017757.py:11: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  weight = np.zeros(6,dtype=np.int)
/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_24613/1732703606.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current

KeyError: 'pwall'

In [ ]:
list_gamma=[0.9,0.7,0.5]
list_taille = [(5,5),(10,10),(10,15),(15,25),(20,20),(25,50),(40,40),(50,50)]
list_taille = [(25,50),(50,50)]
result = experience(1,list_gamma=list_gamma,list_taille=list_taille)


/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_22499/3347125306.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.g = np.zeros((nblignes,nbcolonnes),dtype=np.int)
/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_22499/2737058654.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to

0.3756130420078989 19
0.4558960419963114 24
0.26522670802660286 13
0.341469542006962 18
0.4935037500108592 28
0.4509469589975197 26
0.5038892920129001 30
0.24377237498993054 13
0.4735571670171339 26
0.3300859160081018 18
1.1378702920046635 31
0.6600933330191765 17
1.0906597499852069 29
0.5355104159971233 14
1.1415682499937247 31
1.085590166010661 29
1.1324697500094771 29
1.260782959026983 33
0.7018256670271512 18
1.1417783329961821 30
0.23756999999750406 12
0.27872541701071896 14
0.19731520902132615 10
0.27581166601157747 14
0.26270616697729565 13
0.26616412500152364 15
0.2734936250199098 14
0.2641759169928264 15
0.30626912499428727 15
0.27118254100787453 14
0.545050083019305 14
0.5989573749830015 15
0.54846379099763 14
0.5655313750030473 15
0.5745809580257628 14
0.5949579580046702 15
0.410630708996905 10
0.5578919170075096 14
0.5997041670198087 15
0.43150449998211116 10
0.15380633299355395 8
0.17475337500218302 8
0.17578074999619275 8
0.17075170800671913 8
0.153035416995408 7
0.168651

In [ ]:
stdR = result.std(axis=2)
meanR = result.mean(axis=2)
np.set_printoptions(precision=5, suppress=True)

print(meanR)
print(stdR)

[[[21.5      0.3934 ]
  [26.1      0.98881]]

 [[13.6      0.26334]
  [13.6      0.54273]]

 [[ 7.6      0.16117]
  [ 8.       0.34   ]]]
[[[5.80086 0.09039]
  [6.5643  0.24066]]

 [[1.49666 0.02726]
  [1.85472 0.06384]]

 [[0.91652 0.01828]
  [0.      0.01301]]]


Visualisation

In [ ]:

e.afficheBestAction()
Mafenetre = create_Visualization('test',e, zoom)
Mafenetre.mainloop()

meilleures Actions
['h', 'g', 'f', 'j', 'h', 'h', 'h', 'f', 'g', 'g', 'f', 'g', 'j', 'j', ' ', ' ', ' ', 'f', 'g', 'h', 'g', 'j', 'j', 'h', ' ', 'g', ' ', 'h', 'j', 'j', 'h', 'g', 'j', 'j', ' ', 'h', 'f', 'f', 'f', 'j', 'h', 'h', 'g', ' ', 'j', 'f', 'f', ' ', 'f', 'g']
['u', 'u', 'u', ' ', 'u', 'j', 'r', 'r', 'g', 'f', 'f', ' ', 'f', ' ', 'r', 'j', 'h', ' ', 'g', ' ', 'g', 'g', 'j', 'u', 'h', ' ', 'f', ' ', 'u', ' ', 'u', 'r', ' ', 'g', 'j', 'r', 'h', ' ', 'f', 'u', 'h', 'g', ' ', 'r', 'u', 'h', 'u', ' ', 'r', ' ']
['y', 't', 'r', 'u', 'y', 't', 't', 'r', 'r', 'f', 'y', 'y', 't', 't', 'r', 'h', 'h', 'h', 'g', 'g', 'j', 'f', 'f', 'j', 'y', 'r', ' ', 'y', 'y', 'y', 't', 't', ' ', 'u', 't', 'j', 'r', 'r', 'f', 'g', 'y', 'f', 't', 'f', 'h', 'y', ' ', 't', 'f', 't']
[' ', 't', 'r', 'u', ' ', 'h', 't', 'g', 'r', ' ', 'r', 't', 'r', 'f', 'j', 'j', ' ', 'g', 'g', 'f', 'f', ' ', 'u', ' ', 'f', 'y', 'r', 'u', 'u', ' ', 'y', 't', 't', 'y', 'y', 't', 't', 'h', 'r', 'g', 'f', 'g', 'h', 'h', 'j', 'g

/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_22499/2162625462.py:85: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cost = np.zeros(6,dtype=np.int)
/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_22499/2162625462.py:179: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curren

## Question D 
Pour une grille donné on essayera de piloter à la main quelques trajectoires menant à la case but et on comparera leur coût au coût moyen d’une politique mixte optimale observé sur 20 executions successives de cette politique sur la même grille. On pourra aussi comparer les coûts moyens observés avec la valeur espérée de la politique mixte optimale testée.

In [ ]:

#transitions aleatoires si alea =1 sinon mettre alea=0
alea = 1
#taille de la grille
nb_lines = 15
nb_columns = 15

#cost = np.zeros(6,dtype=np.int)

# valeurs de la grille

weight = np.zeros(6,dtype=np.int)
weight[0] = 1
weight[1] = 2
weight[2] = 4
weight[3] = 8
weight[4] = 16
weight[5] = -1000

seed = random.randint(1,100)
PmurDict=dict()
PmurDict['pmur'] = 0.15 
PmurDict['pblanc'] = 0.45 
PmurDict['pverte'] = 0.1
PmurDict['pbleue'] = 0.1
PmurDict['prouge'] = 0.1
#  pnoire=0.1 mais pas besoin de le specifier c'est la couleur restante
poscav=(0,0)
e = env(nb_columns,nb_lines,weight=weight,alea=alea,PmurDict=PmurDict, seed=seed, posCav=poscav)
e.genere_map_alea()

gamma = 0.9
epsilon = 1

value_iterationG(env=e, gamma= gamma, epsilon=epsilon)
e.afficheBestAction()
Mafenetre = create_Visualization('test',e, zoom)
Mafenetre.mainloop()
print(e.globalcost)
# Mafenetre.destroy()

/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_22499/2472959569.py:11: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  weight = np.zeros(6,dtype=np.int)
/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_22499/3347125306.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curren

meilleures Actions
['j', 'h', 'h', 'h', 'h', 'g', 'h', 'h', 'g', 'g', ' ', 'g', ' ', 'f', 'f']
['h', 'h', 'h', ' ', 'h', 'h', 'g', 'g', 'h', ' ', ' ', 'g', 'f', ' ', 'f']
['j', ' ', 'h', 'j', 'h', 'h', ' ', 'g', 'g', 'g', 'g', 'g', 'f', 'f', 'f']
['j', 'h', 'h', 'h', ' ', 'g', 'g', ' ', ' ', 'f', 'g', 'g', 'f', 'g', 'f']
['h', ' ', 'j', 'h', ' ', 'g', 'g', 'f', 'f', 'g', 'g', ' ', 'f', 'f', 'f']
['h', ' ', 'h', 'j', 'h', 'h', 'g', ' ', 'r', 'f', 'f', 'f', 'f', 'f', ' ']
['j', 'j', ' ', ' ', 'j', 'g', ' ', 'f', 'f', 'g', 'f', 'f', 'r', 'f', 'f']
['j', 'h', 'j', 'u', ' ', 'h', 'f', ' ', 'f', 'r', ' ', 'f', 'f', 'f', 'r']
['u', 'u', 'h', 'u', 'R', 'r', 't', ' ', 'r', 'f', 'r', 'r', ' ', ' ', 'f']
['u', 'y', 'u', 't', 'y', ' ', 'r', 't', ' ', 'f', 'r', 'r', 'r', 'r', ' ']
['u', ' ', 'y', 'y', 'u', 't', 'r', 't', ' ', ' ', 't', 'r', 'r', 'r', 'r']
['y', 'u', ' ', 't', 't', 'y', 'r', ' ', 't', ' ', 't', 't', 't', ' ', 'r']
['j', 'u', 'y', 'y', ' ', 't', 't', 'r', 't', 'f', 'r', 't', 't', 'r'

/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_22499/2162625462.py:85: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cost = np.zeros(6,dtype=np.int)
/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_22499/2162625462.py:179: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curren

-977


# Partie 2 Recherche d'une politique équilibré 

## A
Dans cette partie on considère des grilles contenant environ 15% de cases interdites, ainsi que 20% de
cases rouges et 20% de cases bleues, les couleurs vertes et noires n’étant plus présentes.
a) On cherche une politique optimale équilibrée entre cases rouges et cases bleues pour atteindre l’état but
à partir de l’état initial. Pour cela on utilise des coûts immédiats à deux composantes (c1, c2), de telle sorte
que faire un saut depuis une case blanche engendre un coût immédiat de (1, 1), faire un saut depuis une case
bleue engendre un coût immédiat (2, 0) et faire un saut depuis une case rouge engendre un coût immédiat de
(0, 2). Par exemple, une trajectoire qui fait 22 sauts de cavaliers dont 10 depuis une case blanche, 8 depuis
une case bleue et 4 depuis une case rouge aura pour vecteur coût (10 + 6 × 2, 10 + 4 × 2) = (22, 18), auquel
il faut retirer le bonus final d’atteindre la case but sur chacune des deux composantes. Une trajectoire
traversant autant de cases bleues que de cases rouges aura un vecteur de coûts cumulés équilibré. Pour
évaluer une politique π, on utilise le vecteur (c1(π), c2(π)) qui représentent les coûts espérés actualisés de
la politique π. On souhaite trouver une politique qui optimise le critère suivant :

                                                            min max{c1(π), c2(π)}
                                                             π
                     
Expliquer pourquoi on ne peut utiliser l’itération de la valeur pour la résolution de ce problème. Proposer
alors une approche de résolution reposant sur la formulation d’un MDP bi-objectif et sa résolution par
programmation mathématique pour déterminer une politique mixte optimale.

## B
Tester la résolution pratique du problème de recherche d’une trajectoire équilibrée entre le bleu et le
rouge sur des grilles de différentes tailles avec l’approche proposée à la question précédente. On donnera
ici encore les temps moyens de résolution. Simuler les politiques optimales obtenues en les éxécutant 20
fois sur la même instance. Calculer le vecteur moyen obtenu (c1, c2) qui représente l’espérance de coût
empiriquement observée de la politique optimale selon les deux critères considérés.

## C
Comment modifier l’approche proposée en a) pour déterminer une politique pure optimale au sens du
critère (1) ? Tester cette approche et comparer ses performances à celles trouvées en b) sur les mêmes
instances de problème

# Test

In [ ]:


Mafenetre.mainloop()
